In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_dir='C:\programowanie\python\deep learning\\6\data'
fname=os.path.join(data_dir,'jena_climate_2009_2016.csv')

f=open(fname)
data=f.read()
f.close()

lines=data.split('\n')
header=lines[0].split(',')
lines=lines[1:]

print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [3]:
float_data=np.zeros((len(lines),len(header)-1))

for i ,line in enumerate(lines):
    values=[float(x) for x in line.split(',')[1:]]
    float_data[1,:]=values
    


In [ ]:
temp=float_data[:,1]#C
plt.plot(range(len(temp)),temp)
plt.show

In [ ]:
plt.plot(range(1440),temp[:1440])
plt.show()